In [ ]:

import matplotlib.pyplot as plt
import tifffile as tif
from supramolsim.generate import coordinates_field
from supramolsim import experiments 
from supramolsim.analysis import metrics
import numpy as np
import pandas as pd
import seaborn as sns

# Load image

In [ ]:
experimental_example = "PATH/TO/image.tif"
img_tif = tif.imread(experimental_example)

In [ ]:
plt.imshow(img_tif)
plt.show()

# Approximate particle positions

In [ ]:
# offset in image
background = 0
# pixelsize of image in nanometers
pixelsize = 10
# kernel size for blurring image
sigma = 5
# in pixels, minimal distance expected between centers
min_distance = 10
# pixel value thershold for accepting local maxima
threshold = 0.01

In [ ]:
xyz_relative, image_physical_size = coordinates_field.gen_positions_from_image(
    img_tif,
    mode="localmaxima",
    sigma=sigma,
    background=background,
    threshold=threshold,
    pixelsize=pixelsize,
    min_distance=min_distance)

In [ ]:
plt.imshow(img_tif, cmap="grey")
for rel in xyz_relative:
    rel2 = (rel[0:2] * image_physical_size)
    plt.plot(rel2[1]/pixelsize, rel2[0]/pixelsize, "o", c="r", markersize=2)

# Simulate particles at positions estimated from your image

In [ ]:

structure = "7R5K"
probe_name = "NPC_Nup96_Cterminal_direct"
fluorophore_id = "AF647"
virtual_sample = "square1x1um_randomised" 
modalities = ["STED", ]
_1, experiment = experiments.image_vsample(
    structure=structure,
    probe_name=probe_name,
    fluorophore_id=fluorophore_id,
    virtual_sample=virtual_sample,
    multimodal=modalities,
    particle_positions=xyz_relative,
    run_simulation=False,
    sample_dimensions = [image_physical_size[0],image_physical_size[1], 100]
)
# adjust modality to match your image pixelsize
experiment.update_modality("STED", pixelsize_nm=pixelsize)

In [ ]:
experiment.set_modality_acq("STED", exp_time=0.001, nframes=1)

In [ ]:
images = experiment.run_simulation()

# Circle Fittings on image

### Define the expected dimensions for circle-like objects in the FOV

In [ ]:
# expected parameters of objects
expected_min_radius = 40  # in nanometers
expected_MAX_radius = 70  # in nanometers
min_radius_px = expected_min_radius / pixelsize
MAX_radius_px = expected_MAX_radius / pixelsize
minRadius_round = np.ceil(min_radius_px).astype('int64')
maxRadius_round = np.ceil(MAX_radius_px).astype('int64')
minDist = minRadius_round

### Parameters for fitting multiple circles in image

In [ ]:
# dp, param1 and param2 are user-defined pararmeters
HCparams = dict(dp=1, minDist=maxRadius_round, 
        param1=1, param2=7, minRadius=minRadius_round, maxRadius=maxRadius_round)

## Fit circles on simulated data

In [ ]:
circles_sim, img_blurred_sim, c_params_sim = metrics.get_circles(images["STED"][0], **HCparams)
radii_simulated= []
for (x, y, r) in circles_sim[0]:
    radii_simulated.append((r*pixelsize))


In [ ]:
plt.imshow(img_blurred_sim, cmap="grey")
plt.rcParams['figure.figsize'] = [10,10]
if circles_sim is not None:
    ncircles = circles_sim.shape[1]
    for (x, y, r) in circles_sim[0]:
        circle = plt.Circle((x, y), r, color="r", fill=False, linewidth=2, alpha=0.5)
        plt.gca().add_patch(circle)
        plt.title("Fitted circles. STED simulation")

## Fit circles on experimental data

In [ ]:
circles_exp, img_blurred_exp, c_params_exp = metrics.get_circles(img_tif, **HCparams)
radii_experimental = []
for (x, y, r) in circles_exp[0]:
    radii_experimental.append((r*pixelsize))

In [ ]:
plt.imshow(img_blurred_exp, cmap="grey")
plt.rcParams['figure.figsize'] = [10,10]
if circles_exp is not None:
    ncircles = circles_exp.shape[1]
    for (x, y, r) in circles_exp[0]:
        circle = plt.Circle((x, y), r, color="r", fill=False, linewidth=2, alpha=0.5)
        plt.gca().add_patch(circle)
        plt.title("Fitted circles. STED experimental")

## Plot histograms

In [ ]:

plt.rcParams['figure.figsize'] = [4,4]
df = pd.DataFrame({
    'value': np.concatenate([radii_experimental, radii_simulated]),
    'Condition': ['Experimental'] * len(radii_experimental) + ['Simulated'] * len(radii_simulated)
})
sns.histplot(data=df, x="value", hue="Condition", binrange=[40,70], bins=20, kde=True)
plt.xlabel("Radius (nm)")